In [ ]:
# Enhanced configuration for multiview fusion + output saving
MODEL_KEY = 'qwen25'
MAX_NEW_TOKENS = 128
EXTRA_PROMPT = ''
OUTPUT_DIR = 'outputs'
from utils import ensure_dir
ensure_dir(OUTPUT_DIR)


# Experiment 2: Multiview Fusion (Camera + Map)

Compare action suggestions when the model receives both front camera and bird's-eye map images. Measure delta vs single-view baseline.

In [ ]:
MODEL_KEY = 'qwen25'
MAX_NEW_TOKENS = 128
EXTRA_PROMPT = ''
GROUND_TRUTH = ['LEFT','STOP','SLOW','LEFT','STOP','SLOW']  # example; adjust or load

In [ ]:
from PIL import Image
from utils import list_frame_pairs, build_multiview_prompt, generate_action, parse_action_json
pairs = list_frame_pairs()
len(pairs)

In [ ]:
prompt = build_multiview_prompt(EXTRA_PROMPT)
outputs = []
parsed = []
for cam_path, map_path in pairs:
    cam = Image.open(cam_path).convert('RGB')
    map_img = Image.open(map_path).convert('RGB')
    out = generate_action(MODEL_KEY, [cam, map_img], prompt, max_new_tokens=MAX_NEW_TOKENS)
    outputs.append(out)
    parsed.append(parse_action_json(out) or {})
parsed

In [ ]:
actions = [p.get('action','?') for p in parsed]
actions

In [ ]:
# Persist actions for multiview fusion
from utils import save_text, parse_action_json
if 'parsed' in globals():
    import json, os
    actions = [p.get('action','?') for p in parsed]
    summary = {'actions': actions}
    save_text(os.path.join(OUTPUT_DIR,'experiment2_multiview_summary.json'), json.dumps(summary, indent=2))
    summary
else:
    print('Run generation cells first.')